In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split, StratifiedShuffleSplit
import ml_utilities


C:\Users\David\.conda\envs\Test\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
TESTS_PATH = "./tests/"
INFO_PATH = "./tests/experiments_info"
DATASET_TEST_PATH = 'DBs/PenDigits/pendigits_te.txt'
DATASET_PATH = 'DBs/PenDigits/pendigits_tr.txt'

In [5]:
def read_data():
    feat_count = 16
    data_path = DATASET_PATH # Impostare il percorso corretto
    patterns, labels = ml_utilities.load_labeled_dataset_from_txt(data_path, feat_count)
    test_path = DATASET_TEST_PATH
    x_test = ml_utilities.load_unlabeled_dataset_from_txt(test_path, feature_count)
    return patterns, labels, x_test

In [4]:
from sklearn.preprocessing import StandardScaler


def normalize_data(x_train, x_test):
    scaler = StandardScaler()
    scaler.fit(dataset_patterns)
    x_train_normalized = scaler.fit_transform(x_train)
    x_test_normalized = scaler.fit_transform(x_test)

    return x_train_normalized, x_test_normalized

In [37]:
# def feature_extractor(x_grid):
#     SIZE = x_grid.shape[1]
#     activation = 'sigmoid'
#     feature = Sequential()
#
#     feature.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (SIZE, SIZE, 1)))
#     feature.add(BatchNormalization())
#
#     feature.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
#     feature.add(BatchNormalization())
#     feature.add(MaxPool2D())
#
#     feature.add(Flatten())
#
#     return feature

In [65]:
model_params = {
    'SVC': {
        'model': SVC( ),
        'params' : {
            'C': [2**i for i in range(-5,16)] + [1],  #Regularization parameter. Providing only two as SVM is slow
            'kernel': ['rbf','linear'],
            'gamma': [2**i for i in range(-5,16)] + [0.00009111627561154887],
            'class_weight':['balanced', None],
            'degree' : [1,2,3,4,5,6]
        }
    },
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'params' : {
            'n_neighbors': [1,3,5,7,9,11,13,15,17,19,21,23,25,27],
            'weights': ['uniform','distance'],
            'algorithm': ["ball_tree","kd_tree","brute"],
            'metric': ["minkowski","euclidean","l1", "l2","manhattan"],

        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [2**i for i in range(-5,16)] + [1],  #Regularization. . Providing only two as LR can be slow
            'penalty': ['l1', 'l2']
        }
    }
}
scores=[]

In [3]:
params = {
        'model': SVC(),
        'params' : {
            'C': [2**i for i in range(-5,16)],  #Regularization parameter. Providing only two as SVM is slow
            'kernel': ['rbf'],
            'gamma': [2**i for i in range(-15,4)] + [0.00009111627561154887],
        }
}
scores=[]

In [6]:
# Caricamento del dataset
feature_count = 16
dataset_path = 'DBs/PenDigits/pendigits_tr.txt'  # Impostare il percorso corretto

dataset_patterns, dataset_labels = ml_utilities.load_labeled_dataset_from_txt(dataset_path, feature_count)
print('Shape dataset:', dataset_patterns.shape)
print('Shape labels:', dataset_labels.shape)

Shape dataset: (442, 16)
Shape labels: (442,)


In [48]:
!conda create -n xgboost_env -c nvidia -c rapidsai py-xgboost cudatoolkit=10.2
np.random.seed(42)

x_train_raw, y_train, x_test_raw = read_data()
x_train, x_test = normalize_data(x_train_raw,x_test_raw)
for data_test_size in [0.2]:
    for folds in [1]:
        cross_val = StratifiedShuffleSplit(n_splits=folds, test_size=data_test_size, random_state=42)
        xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic', nthread=1,tree_method='gpu_hist')
        params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.02, 0.05]
        }
        for model_name, mp in model_params.items():
            # pipe = Pipeline([('classifier' , mp['model'])])
            grid =  RandomizedSearchCV(xgb,
                                       param_distributions=params,
                                       cv=cross_val,
                                       n_jobs=16,
                                       n_iter=2,
                                       return_train_score=False,
                                       random_state=42)

            grid.fit(x_train,y_train)
            scores.append({
                'data_test_size': data_test_size,
                'folds': folds,
                'model': model_name,
                'best_score': grid.best_score_,
                'mean_test_score': grid.cv_results_['mean_test_score'],
                'best_params': grid.best_params_,
            })

        df = pd.DataFrame(scores,columns=['data_test_size','folds','model','best_score','mean_test_score','best_params',])

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


C:\Users\David\.conda\envs\Test\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 2.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\David\.conda\envs\Test\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\David\.conda\envs\Test\lib\site-packages\xgboost\core.py", line 506, in inner_f
    return f(**kwargs)
  File "C:\Users\David\.conda\envs\Test\lib\site-packages\xgboost\sklearn.py", line 1250, in fit
    self._Booster = train(
  File "C:\Users\David\.conda\envs\Test\lib\site-packag

XGBoostError: [12:37:58] c:\windows\temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\common\common.h:157: XGBoost version not compiled with GPU support.

In [15]:
np.random.seed(42)

x_train, y_train, x_test = read_data()
x_train, x_test = normalize_data(x_train,x_test)
params = {
            'C': [2**i for i in range(-5,16)],  #Regularization parameter. Providing only two as SVM is slow
            'kernel': ['rbf'],
            'gamma': [2**i for i in range(-15,4)] + [0.00009111627561154887],

}
svc = SVC()
scores = []

In [28]:


for data_test_size in [0.3, 0.4, 0.5]:
    for folds in [9,10]:
        cross_val = StratifiedShuffleSplit(n_splits=folds, test_size=data_test_size, random_state=42)
        grid =  GridSearchCV(svc,
                             param_grid=params,
                             cv=cross_val)

        grid.fit(x_train,y_train)
        scores.append({
            'data_test_size': data_test_size,
            'folds': folds,
            'best_score': grid.best_score_,
            'mean_test_score': grid.cv_results_['mean_test_score'],
            'best_params': grid.best_params_,
        })
        print(f"fit completed")
    d9 = pd.DataFrame(scores,columns=['data_test_size','folds','best_score','mean_test_score','best_params'])
    print(f"data_test_size= {data_test_size} and folds = {folds}")
    print(f"fit completed")

fit completed
fit completed
data_test_size= 0.3 and folds = 10
fit completed
fit completed
fit completed
data_test_size= 0.4 and folds = 10
fit completed
fit completed
fit completed
data_test_size= 0.5 and folds = 10
fit completed


In [32]:
dataframe = pd.read_pickle(TESTS_PATH+"/Training")
dataframe.sort_values(by='best_score', ascending=False)
dataframe

data_test_size  folds                model  best_score  \
0              0.10      1                  SVC    0.933333   
1              0.10      1           KNeighbors    0.888889   
2              0.10      1  logistic_regression    0.777778   
3              0.10      2                  SVC    0.866667   
4              0.10      2           KNeighbors    0.855556   
..              ...    ...                  ...         ...   
508            0.05      8           KNeighbors    0.864130   
509            0.05      8  logistic_regression    0.782609   
510            0.05      9                  SVC    0.864734   
511            0.05      9           KNeighbors    0.855072   
512            0.05      9  logistic_regression    0.782609   

                                       mean_test_score  \
0    [0.15555555555555556, 0.3111111111111111, 0.15...   
1    [0.7555555555555555, 0.7555555555555555, 0.711...   
2    [0.13333333333333333, 0.4666666666666667, 0.26...   
3    [0.14444444444444443, 0.28888888888888886, 0.1...   
4    [0.7333333333333334, 0.7333333333333334, 0.766...   
..                                                 ...   
508  [0.7663043478260869, 0.7663043478260869, 0.826...   
509  [0.13043478260869565, 0.4782608695652174, 0.35...   
510  [0.08695652173913043, 0.2657004830917874, 0.08...   
511  [0.7536231884057971, 0.7536231884057971, 0.806...   
512  [0.13043478260869565, 0.4734299516908213, 0.34...   

                                           best_params  
0    {'C': 0.125, 'class_weight': 'balanced', 'degr...  
1    {'algorithm': 'ball_tree', 'metric': 'minkowsk...  
2                          {'C': 0.5, 'penalty': 'l2'}  
3    {'C': 1, 'class_weight': 'balanced', 'degree':...  
4    {'algorithm': 'ball_tree', 'metric': 'minkowsk...  
..                                                 ...  
508  {'algorithm': 'ball_tree', 'metric': 'minkowsk...  
509                          {'C': 1, 'penalty': 'l1'}  
510  {'C': 1, 'class_weight': None, 'degree': 1, 'g...  
511  {'algorithm': 'ball_tree', 'metric': 'minkowsk...  
512                          {'C': 1, 'penalty': 'l1'}  

[513 rows x 6 columns]

In [25]:
dd

data_test_size  folds  best_score  \
0              0.3      6    0.845865   
1              0.3      7    0.848550   
2              0.3      6    0.845865   
3              0.3      6    0.845865   
4              0.3      7    0.848550   
5              0.3      8    0.846805   
6              0.3      8    0.846805   
7              0.4      6    0.834275   
8              0.4      7    0.830508   
9              0.4      8    0.831215   
10             0.4      8    0.831215   
11             0.5      6    0.811463   
12             0.5      7    0.812540   
13             0.5      8    0.813348   
14             0.5      8    0.813348   

                                      mean_test_score  \
0   [0.3659147869674186, 0.3659147869674186, 0.365...   
1   [0.3619763694951665, 0.3619763694951665, 0.361...   
2   [0.3659147869674186, 0.3659147869674186, 0.365...   
3   [0.3659147869674186, 0.3659147869674186, 0.365...   
4   [0.3619763694951665, 0.3619763694951665, 0.361...   
5   [0.36184210526315785, 0.36184210526315785, 0.3...   
6   [0.36184210526315785, 0.36184210526315785, 0.3...   
7   [0.37288135593220334, 0.37288135593220334, 0.3...   
8   [0.37207425343018563, 0.37207425343018563, 0.3...   
9   [0.3707627118644068, 0.3707627118644068, 0.370...   
10  [0.3707627118644068, 0.3707627118644068, 0.370...   
11  [0.1711915535444947, 0.1711915535444947, 0.171...   
12  [0.16548157724628312, 0.16548157724628312, 0.1...   
13  [0.1923076923076923, 0.1923076923076923, 0.192...   
14  [0.1923076923076923, 0.1923076923076923, 0.192...   

                                  best_params  
0   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
1   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
2   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
3   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
4   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
5   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
6   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
7   {'C': 8, 'gamma': 0.125, 'kernel': 'rbf'}  
8   {'C': 8, 'gamma': 0.125, 'kernel': 'rbf'}  
9   {'C': 8, 'gamma': 0.125, 'kernel': 'rbf'}  
10  {'C': 8, 'gamma': 0.125, 'kernel': 'rbf'}  
11  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}  
12  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}  
13  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}  
14  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}

In [29]:
d9

data_test_size  folds  best_score  \
0              0.3      6    0.845865   
1              0.3      7    0.848550   
2              0.3      6    0.845865   
3              0.3      6    0.845865   
4              0.3      7    0.848550   
5              0.3      8    0.846805   
6              0.3      8    0.846805   
7              0.4      6    0.834275   
8              0.4      7    0.830508   
9              0.4      8    0.831215   
10             0.4      8    0.831215   
11             0.5      6    0.811463   
12             0.5      7    0.812540   
13             0.5      8    0.813348   
14             0.5      8    0.813348   
15             0.3      9    0.842941   
16             0.3     10    0.836842   
17             0.4      9    0.827370   
18             0.4     10    0.823729   
19             0.5      9    0.810458   
20             0.5     10    0.812217   

                                      mean_test_score  \
0   [0.3659147869674186, 0.3659147869674186, 0.365...   
1   [0.3619763694951665, 0.3619763694951665, 0.361...   
2   [0.3659147869674186, 0.3659147869674186, 0.365...   
3   [0.3659147869674186, 0.3659147869674186, 0.365...   
4   [0.3619763694951665, 0.3619763694951665, 0.361...   
5   [0.36184210526315785, 0.36184210526315785, 0.3...   
6   [0.36184210526315785, 0.36184210526315785, 0.3...   
7   [0.37288135593220334, 0.37288135593220334, 0.3...   
8   [0.37207425343018563, 0.37207425343018563, 0.3...   
9   [0.3707627118644068, 0.3707627118644068, 0.370...   
10  [0.3707627118644068, 0.3707627118644068, 0.370...   
11  [0.1711915535444947, 0.1711915535444947, 0.171...   
12  [0.16548157724628312, 0.16548157724628312, 0.1...   
13  [0.1923076923076923, 0.1923076923076923, 0.192...   
14  [0.1923076923076923, 0.1923076923076923, 0.192...   
15  [0.3625730994152046, 0.3625730994152046, 0.362...   
16  [0.3646616541353383, 0.3646616541353383, 0.364...   
17  [0.3691148775894539, 0.3691148775894539, 0.369...   
18  [0.3700564971751413, 0.3700564971751413, 0.370...   
19  [0.1855203619909502, 0.1855203619909502, 0.185...   
20  [0.20407239819004525, 0.20407239819004525, 0.2...   

                                  best_params  
0   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
1   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
2   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
3   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
4   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
5   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
6   {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
7   {'C': 8, 'gamma': 0.125, 'kernel': 'rbf'}  
8   {'C': 8, 'gamma': 0.125, 'kernel': 'rbf'}  
9   {'C': 8, 'gamma': 0.125, 'kernel': 'rbf'}  
10  {'C': 8, 'gamma': 0.125, 'kernel': 'rbf'}  
11  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}  
12  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}  
13  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}  
14  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}  
15  {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
16  {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
17  {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
18  {'C': 2, 'gamma': 0.125, 'kernel': 'rbf'}  
19  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}  
20  {'C': 1, 'gamma': 0.125, 'kernel': 'rbf'}

In [182]:
dataframe['score'] = 0
# for SVC
datawork = dataframe.loc[dataframe['model'] == "SVC"]
dataframe['score'].update(datawork['best_params'].map(lambda x: SVC(**x).fit(x_train, y_train).score(x_train, y_train)))
# for KNN
datawork = dataframe.loc[dataframe['model'] == "KNeighbors"]
dataframe['score'].update(datawork['best_params'].map(lambda x: KNeighborsClassifier(**x).fit(x_train, y_train).score(x_train, y_train)))# for KNN
dataframe.sort_values(by='best_score', ascending=False)
datacopy = dataframe
datacopy = datacopy[(dataframe.data_test_size > 0.25) & (dataframe.data_test_size < 0.7)]
datacopy = datacopy[(dataframe.folds > 5)]
datacopy = datacopy[(dataframe.model != 'logistic_regression')]

C:\Users\David\AppData\Local\Temp\ipykernel_2824\2019163029.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datacopy = datacopy[(dataframe.folds > 5)]
C:\Users\David\AppData\Local\Temp\ipykernel_2824\2019163029.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datacopy = datacopy[(dataframe.model != 'logistic_regression')]


In [31]:
dataframe['total_mean'] = 0
dataframe['total_mean'].update(datawork['mean_test_score'].map(lambda m: m.mean()))# for KNN
dataframe

NameError: name 'datawork' is not defined

In [35]:
databest = d9.sort_values(by='best_score', ascending=True)
best_classifier = SVC(**databest['best_params'][0])
# Addestramento del classificatore
best_classifier.fit(x_train, y_train)
# Calcolo delle prediction
predictions = best_classifier.predict(x_test)
predictions

array([2., 4., 0., ..., 0., 8., 2.])

In [34]:
databest = d9.sort_values(by='best_score', ascending=True)
best_classifier = SVC(**databest['best_params'][0])
best_classifier

SVC(C=2, gamma=0.125)

In [36]:
result_path = 'Es3Predictions.txt'
with open(result_path, "w") as f:
    for prediction in predictions:
        f.write(str(int(prediction)) + '\n')
print('Salvataggio delle prediction riuscito')

Salvataggio delle prediction riuscito
